In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import preprocess_input
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.segmentation import mark_boundaries

# -----------------------------
# Paths & Classes
# -----------------------------
model_path = "/kaggle/input/mymodel/inceptionv3_model1.keras"
base_dir = "/kaggle/input/testdata/stomachTest"
class_names = ['polyps', 'esophagitis', 'ulcerative-colitis']

# -----------------------------
# Load model
# -----------------------------
model = load_model(model_path)

# -----------------------------
# LIME setup
# -----------------------------
explainer = lime_image.LimeImageExplainer()
segmenter = SegmentationAlgorithm('quickshift', kernel_size=1, max_dist=200, ratio=0.2)

# -----------------------------
# Plot setup
# -----------------------------
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 12))
fig.suptitle("LIME Explanations for Stomach Disease Classification", fontsize=16)

# -----------------------------
# Loop through each class
# -----------------------------
for row, class_name in enumerate(class_names):
    class_path = os.path.join(base_dir, class_name)
    image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))][:2]

    for col, img_name in enumerate(image_files):
        img_path = os.path.join(class_path, img_name)

        # Load and preprocess
        img_bgr = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img_rgb, (224, 224))
        input_img_batch = np.expand_dims(preprocess_input(img_resized.astype(np.float32)), axis=0)

        # Predict class
        preds = model.predict(input_img_batch)
        pred_class = np.argmax(preds[0])

        # LIME explanation
        explanation = explainer.explain_instance(
            image=img_resized,
            classifier_fn=lambda x: model.predict(preprocess_input(x.astype(np.float32))),
            segmentation_fn=segmenter,
            top_labels=3,
            hide_color=0,
            num_samples=500  # safer for Kaggle
        )

        temp, mask = explanation.get_image_and_mask(
            label=pred_class,
            positive_only=False,
            hide_rest=False,
            num_features=10,
            min_weight=0.0
        )

        axes[row, col].imshow(mark_boundaries(temp / 255.0, mask))
        axes[row, col].set_title(f"{class_name.title()} - Image {col+1}")
        axes[row, col].axis('off')

plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()
